In [1]:
#Imports
import sys
sys.path.append('../../python/')
import NGC5533_functions as nf

import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import lmfit as lm
import dataPython as dp

from datetime import datetime
import time as t

In [2]:
starttime = t.time()

data = dp.getXYdata_wXYerr('../data/100kpc_data.txt')
r_dat = np.asarray(data['xx'])
v_dat = np.asarray(data['yy'])
v_err0 = np.asarray(data['ex'])
v_err1 = np.asarray(data['ey'])

#change r_dat so it's strictly increasing
r_dat, v_dat, v_err0, v_err1 = (list(a) for a in zip(*sorted(zip(r_dat, v_dat, v_err0, v_err1))))
#converting v_err1 to an array
v_err1 = np.asarray(v_err1)
#Express as weights
weighdata = 1/v_err1

In [3]:
#keeping others but holding them constant
def f(r,M,rc,h_rho00):
     return np.sqrt(nf.bh_v(r,M,load=True,path='../')**2 
                    + nf.h_v(r,rc,h_rho00,load=True,path='../')**2 
                    + nf.b_v(r,load=True,path='../')**2 
                    + nf.d_v(r,load=True,path='../')**2)
#Only Black Hole and Halo
def bh_h(r,M,rc,h_rho00):
    return np.sqrt(nf.bh_v(r,M,load=True,path='../')**2 + nf.h_v(r,rc,h_rho00,load=True,path='../')**2)

In [4]:
#Fit, Keeping rest constant

#Setup
l_mod = lm.Model(f)
l_params = l_mod.make_params()
#Black Hole Params
l_params.add('M', value=nf.Mbh_def, min=0) #Mass
#Halo Params
l_params.add('rc', value=nf.h_rc, min=0)   #Radius (magnitude)
l_params.add('h_rho00', value=nf.hrho00_c, min=0) #Density (halo)

#Do fit
l_fit = l_mod.fit(v_dat,l_params,r=r_dat,weights=weighdata)

../../python\NGC5533_functions.py:308: RuntimeWarning: overflow encountered in cosh
  return d_durho0(r, h, d_rho00)*np.power(np.cosh(z/z0(h)), -2)
../../python\NGC5533_functions.py:311: RuntimeWarning: invalid value encountered in double_scalars
  return 2*(ss.ellipk(d_px(r,u,xi)) - ss.ellipe(d_px(r,u,xi)))/(np.pi*np.sqrt(r*u*d_px(r,u,xi)))
C:\Users\Cirdan\Anaconda3\lib\site-packages\scipy\integrate\quadpack.py:385: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)
C:\Users\Cirdan\Anaconda3\lib\site-packages\scipy\integrate\quadpack.py:385: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuit

In [ ]:
#Fit, BH and H only

#Setup
bhh_mod = lm.Model(bh_h)
bhh_params = bhh_mod.make_params()
#Black Hole Params
bhh_params.add('M', value=nf.Mbh_def, min=0) #Mass
#Halo Params
bhh_params.add('rc', value=nf.h_rc, min=0)   #Radius (magnitude)
bhh_params.add('h_rho00', value=nf.hrho00_c, min=0) #Density (halo)
#Do fit
bhh_fit = bhh_mod.fit(v_dat,bhh_params,r=r_dat,weights=weighdata)

In [ ]:
#Define curves to plot

l_dict = l_fit.best_values
l_M = l_dict['M']
l_rc = l_dict['rc']
l_h_rho00 = l_dict['h_rho00']
l_curve = f(r_dat,l_M,l_rc,l_h_rho00)

bhh_dict = bhh_fit.best_values
bhh_M = bhh_dict['M']
bhh_rc = bhh_dict['rc']
bhh_h_rho00 = bhh_dict['h_rho00']

In [ ]:
fig = plt.figure(figsize=(9.0,8.0))

plt.xlim(0,125)

plt.errorbar(r_dat,v_dat,yerr=v_err1,fmt='bo',label='Data')
plt.plot(r_dat,l_curve,'b-',label='Fit with All Components')
plt.plot(r_dat,bh_h(r_dat,l_M,l_rc,l_h_rho00),'b--',label='Black Hole + Halo, from All Components')
plt.plot(r_dat,bh_h(r_dat,nf.Mbh_def,nf.h_rc,nf.hrho00_c),'g-.',label='Black Hole + Halo (Defaults)')
plt.plot(r_dat,bh_h(r_dat,bhh_M,bhh_rc,bhh_h_rho00),label='Fit Black Hole + Halo Only')

plt.legend()
plt.show()

print('Fit information for single-component fit')
l_fit

In [ ]:
fig = plt.figure(figsize=(9.0,8.0))

plt.xlim(0,25)
plt.ylim(0,400)

plt.errorbar(r_dat,v_dat,yerr=v_err1,fmt='bo',label='Data')
plt.plot(r_dat,l_curve,'b-',label='Fit with All Components')
plt.plot(r_dat,bh_h(r_dat,l_M,l_rc,l_h_rho00),'b--',label='Black Hole + Halo, from All Components')
plt.plot(r_dat,bh_h(r_dat,nf.Mbh_def,nf.h_rc,nf.hrho00_c),'g-.',label='Black Hole + Halo (Defaults)')
plt.plot(r_dat,bh_h(r_dat,bhh_M,bhh_rc,bhh_h_rho00),label='Fit Blakc Hole + Halo Only')

plt.legend()
plt.show()

print('Fit information for two-component fit')
bhh_fit

In [ ]:
endtime = t.time()
print('Total Time:')
runtime = endtime - starttime

if runtime<=3600:
    minutes, runtime= divmod(runtime, 60)
    print(str(minutes)+'min', str(runtime)+'s')

elif runtime>3600 and runtime<=86400:
    minutes = runtime/60
    hours, minutes= divmod(minutes, 60)
    print(str(hours)+'hrs', str(round(minutes))+'min')

elif runtime>86400:
    hours = runtime/3600
    days, hours= divmod(hours, 24)
    print(str(days)+'days', str(round(hours))+'hrs')